In [ ]:
import requests
import pandas as pd
import json
from tqdm import tqdm
import codecs

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko'
}

In [ ]:
linkFrontEncoded = 'uggcf://jjj.pbafhzreercbegf.bet/pnef/engvatf-qngn/'
linkFront = codecs.encode(linkFront, 'rot_13')

In [ ]:
carData=[]
for i in tqdm(range(7200,9200)):
    try:
        #print(i)
        #get car data
        carResponse = requests.get("".join([linkFront,str(i)]))
        carResponse.raise_for_status() # raise exception if invalid response
        carData.extend(carResponse.json())
    except requests.exceptions.RequestException as e:
        print(e)
        #skipList.append(i)
        continue

In [ ]:
from datetime import datetime
# Get current date
date_object = datetime.now()
# convert object to the format we want
formatted_date = date_object.strftime('%m%d%Y')
formatted_date

In [ ]:
import pickle
pickle.dump( carData, open( formatted_data + "_carData.p", "wb" ) )

In [ ]:
carData[60][2]['items'][0]['items'][0]

In [ ]:
vehicleData = []
for c in carData:
    d = {}
    d['model'] = c[0]['compareInfo']['name']
    d['trim'] = c[0]['compareInfo']['subName']
    d['modelYearId'] = c[0]['compareInfo']['modelYearId']
    d['overallScore'] = c[1]['items'][0]['value']
    d['roadTest'] = c[1]['items'][1]['value']
    d['AEBCity'] = c[3]['items'][0]['items'][0]['items'][1]['value']
    d['AEBHighway'] = c[3]['items'][0]['items'][0]['items'][2]['value']
    
    vehicleData.append(d)

In [ ]:
pd.DataFrame(vehicleData).to_excel('CRd')

In [ ]:
jsonFiles = []
jsonFiles.append('smallcarreliability.txt')
jsonFiles.append('evreliability.txt')
jsonFiles.append('pickupreliability.txt')
jsonFiles.append('minivanreliability.txt')
jsonFiles.append('sedanreliability.txt')
jsonFiles.append('sportscarreliability.txt')
jsonFiles.append('suvreliability.txt')
jsonFiles.append('wagonreliability.txt')
jsonFiles.append('luxurycarreliability.txt')
jsonFiles.append('convertablesreliability.txt')

In [ ]:
import json

In [ ]:
data = json.load(open(jsonFiles[0]))

In [ ]:
data = []
for f in jsonFiles:
    s = json.load(open(f))
    data.extend(s['response'])

In [ ]:
data[0]['modelYearReliabilityRatings']['predictedReliabilityScore']

In [ ]:
relData = []
for dm in data:
    if 'modelYearReliabilityRatings' in list(dm.keys()):
        if 'predictedReliabilityScore' in list(dm['modelYearReliabilityRatings'].keys()):
            d = {}
            d['reliability'] = dm['modelYearReliabilityRatings']['predictedReliabilityScore']
            d['modelYearId'] = dm['modelYearId']
    if 'modelYearOwnerSatisfactionRatings' in list(dm.keys()):
        if 'predictedWouldBuyAgainPercent' in list(dm['modelYearOwnerSatisfactionRatings'].keys()):
            d['ownerSat'] = dm['modelYearOwnerSatisfactionRatings']['predictedWouldBuyAgainPercent']
    relData.append(d)

In [ ]:
pd.DataFrame(relData).to_excel('relData.xlsx')

In [ ]:
def flattenjson(y):
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            if 'label' in list(x.keys()) and 'value' in list(x.keys()):
                out[x['label']] = x['value']
            else:
                for a in x:
                    flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [ ]:
flat = []
for i in data:
    flat.append(flattenjson(i))

In [ ]:
pd.DataFrame(flat).to_excel('rel.xlsx')

In [ ]:
flatCarData = []
for i in carData:
    flatCarData.append(flattenjson(i))

In [ ]:
def getYear(row):
    s = row['0_compareInfo_name'][:4]
    return s

In [ ]:
def getMake(row):
    s = row['0_compareInfo_name'][5:]
    s = s.split(' ')
    if len(s)==4:
        make = s[0]+' '+s[1]
    elif len(s)==3:
        make = s[0]+' '+s[1]
    else:
        make = s[0]
    return make

In [ ]:
def getModel(row):
    s = row['0_compareInfo_name'][5:]
    s = s.split(' ')
    if len(s)==4:
        make = s[2]+' '+s[3]
    elif len(s)==3:
        make = s[2]
    else:
        make = s[1]
    return make

In [ ]:
d = pd.DataFrame(flatCarData)
d['Year'] = d.apply(getYear,axis=1)
d['Make'] = d.apply(getMake,axis=1)
d['Model'] = d.apply(getModel, axis=1)
d.to_excel('data.xlsx')